In [1]:
from __future__ import print_function
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout
current_dir ='/home/sanderroet/scripts/test_show/'

gro = app.GromacsGroFile(current_dir+'vis-md.gro')
top = app.GromacsTopFile(current_dir+'topol.top', periodicBoxVectors=gro.getPeriodicBoxVectors(),
                        includeDir='/home/sanderroet/top')




In [ ]:
system = top.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.1*unit.nanometer,
        constraints=app.HBonds, rigidWater=False, ewaldErrorTolerance=0.0005)
integrator = mm.LangevinIntegrator(310*unit.kelvin, 1.0/unit.picosecond, 2.0*unit.femtoseconds)
system.addForce(mm.MonteCarloBarostat(1*unit.atmospheres, 310*unit.kelvin, 25))

#platform = mm.Platform.getPlatformByName('CPU')

simulation =app.Simulation(top.topology, system, integrator)#,platform)
simulation.context.setPositions(gro.positions)

In [ ]:
%%time
print('Minimizing...')
simulation.minimizeEnergy()
print('Done minimizing')

Minimizing...


In [ ]:
dof = 0
system = simulation.system
dofs_from_particles = 0
for i in range(system.getNumParticles()):
    if system.getParticleMass(i) > 0* unit.dalton:
        dofs_from_particles += 3
dofs_from_constraints = system.getNumConstraints()
dofs_from_motion_removers = 0
if any(type(system.getForce(i)) == mm.CMMotionRemover for i in range(system.getNumForces())):
    dofs_from_motion_removers += 3
dof = dofs_from_particles - dofs_from_constraints - dofs_from_motion_removers
print (dof) 
print (dofs_from_particles)
print (dofs_from_constraints)
print (dofs_from_motion_removers)
#print dof, "=", dofs_from_particles, "-", dofs_from_constraints, "-", dofs_from_motion_removers

In [ ]:
simulation.context.setVelocitiesToTemperature(310*unit.kelvin)

In [ ]:
state = simulation.context.getState(getVelocities=True, getEnergy=True)
R = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA

ke = state.getKineticEnergy()

print(ke)
print(2 * ke / dof / R)
print(310*unit.kelvin/2*dof*R)
print(ke/(310*unit.kelvin/2*dof*R))
print()

In [ ]:

simulation.reporters.append(app.StateDataReporter(stdout, 1, step=True, 
    potentialEnergy=True, temperature=True, progress=True, remainingTime=True, 
    speed=True, totalSteps=1000, separator='\t'))

print('Equilibrating...')
simulation.step(100)

In [ ]:
%%time
simulation.reporters.append(app.PDBReporter(current_dir+'new_trajectory.pdb', 1000))
simulation.reporters.append(app.DCDReporter(current_dir+'new_trajectory.dcd', 10))
simulation.reporters.append(app.StateDataReporter(stdout, 10, step=True, 
    potentialEnergy=True, temperature=True, progress=True, remainingTime=True, 
    speed=True, totalSteps=1000, separator='\t'))


In [ ]:
%%time

#print('Running Production...')
#simulation.step(1000)
#print('Done!')